# Shadow casting
[More information on the shadow casting analysis](/A3_Massing/Process/Shadow%20analysis/Shadow%20analysis/)  


## 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
from ladybug.sunpath import Sunpath
import pandas as pd

## 1. Import Meshes (context + envelope)

### 1.1. Load Meshes

In [2]:
envelope_path = os.path.relpath('../data/envelope.obj')
context_path = os.path.relpath('../data/context_with_park2.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!


True
False


### 1.2. Visualize Meshes (with pyvista)

In [3]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]

## 2. Import Lattice (envelope)

### 2.1. Load the Envelope Lattice

In [4]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/envelope_lowres_10.8.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

### 2.2. Visualize the Context Mesh + Envelope Lattice

In [5]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(786.0557649294859, 708.6393599294859, 740.9666770318022),
 (65.08283250000001, -12.333572500000002, 19.993744602316283),
 (0.0, 0.0, 1.0)]

## 3. Sun Vectors

### 3.1. Compute Sun Vectors

In [6]:
# initiate sunpath
sp = Sunpath(longitude=4.3571, latitude=52.0116)

# define sun hours : A list of hours of the year for each sun vector
# there are 8760 hours in a year, so the following integers refer to specific hours throughout the year
hoys = []
sun_vectors = []
day_multiples = 30
for d in range(365):
    if d%day_multiples==0:
        for h in range(24):
            i = d*24 + h
            # compute the sun object
            sun = sp.calculate_sun_from_hoy(i)
            # extract the sun vector
            sun_vector = sun.sun_vector.to_array()
            # apparantly, if the Z component of sun vector is positive, 
            # it is under the horizon 
            if sun_vector[2] < 0.0:
                hoys.append(i)
                sun_vectors.append(sun_vector)
                
sun_vectors = np.array(sun_vectors)
# compute the rotation matrix 
Rz = tm.transformations.rotation_matrix(np.radians(36.324), [0,0,1])
# Rotate the sun vectors to match the site rotation
sun_vectors = tm.transform_points(sun_vectors, Rz)
print(sun_vectors.shape)

(154, 3)


In [7]:
full_lattice = envelope_lattice * 0 + 1

In [8]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
full_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# add the sun locations, color orange
p.add_points( - sun_vectors * 300, color='#ffa500')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(987.1796013214682, 980.6322311914033, 1113.9398701989232),
 (1.8922800195791183, -4.65509011048573, 128.65254889703414),
 (0.0, 0.0, 1.0)]

## 4. Compute Intersection of Sun Rays with Context Mesh (other direction)

### 4.1. Preparing the List of Ray Directions and Origins

In [9]:
# constructing the sun direction (other direction) from the sun vectors in a numpy array
sun_dirs = np.array(sun_vectors)
# exract the centroids of the envelope voxels

vox_cens = full_lattice.centroids
 
ray_dir = []
ray_src = []
for v_cen in vox_cens:
    for s_dir in sun_dirs:
        ray_dir.append(s_dir)
        ray_src.append(v_cen)
# converting the list of directions and sources to numpy array
ray_dir = np.array(ray_dir)
ray_src = np.array(ray_src)

print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sun_dirs.shape)
print("number of rays to be shooted :",ray_src.shape)

number of voxels to shoot rays from : (540, 3)
number of rays per each voxel : (154, 3)
number of rays to be shooted : (83160, 3)


### 4.2. Computing the Intersection

In [10]:
# computing the intersections 
tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)

## 5. Aggregate Simulation Result in the Shadow Casting Lattice

### 5.1. Compute the percentage of time that each ray intersects a voxel 

In [13]:
# initializing the hits list full of zeros
hits = [0]*len(ray_dir)
# setting the rays that had an intersection to 1
for id in ray_id:
    hits[id] = 1

sun_count = len(sun_dirs)
vox_count = len(vox_cens)
# initiating the list of ratio
vox_shadow_casting = []
# iterate over the voxels
for v_id in range(vox_count):
    # counter for the intersection
    int_count = 0
    # iterate over the sun rays
    for s_id in range(sun_count):
        # computing the ray id from voxel id and sun id
        r_id = v_id * sun_count + s_id

        # summing the intersections
        int_count += hits[r_id]
    
    # computing the percentage of the rays that DID NOT have 
    # an intersection
    shadow_casting = int_count/sun_count

    # add the ratio to list
    vox_shadow_casting.append(shadow_casting)


hits = np.array(hits)
vox_shadow_casting = np.array(vox_shadow_casting)

print(vox_shadow_casting)

[0.         0.21428571 0.36363636 0.43506494 0.         0.18181818
 0.40909091 0.47402597 0.         0.21428571 0.42857143 0.53896104
 0.         0.23376623 0.44805195 0.56493506 0.         0.25324675
 0.5        0.57792208 0.         0.27272727 0.55194805 0.6038961
 0.         0.2987013  0.61688312 0.62987013 0.         0.37662338
 0.59090909 0.62337662 0.         0.3961039  0.56493506 0.58441558
 0.         0.68831169 0.44155844 0.38961039 0.         0.17532468
 0.32467532 0.38961039 0.         0.17532468 0.37662338 0.44155844
 0.         0.18831169 0.4025974  0.5        0.         0.2012987
 0.40909091 0.53246753 0.         0.22077922 0.47402597 0.55844156
 0.         0.24025974 0.53896104 0.62987013 0.         0.43506494
 0.58441558 0.63636364 0.         1.         0.57792208 0.57142857
 0.         0.30519481 0.34415584 0.37662338 0.         0.22727273
 0.34415584 0.37012987 0.         0.17532468 0.32467532 0.38961039
 0.         0.17532468 0.33766234 0.41558442 0.         0.188311

### 5.2. Store shadow casting information in a Lattice

In [14]:
# getting the condition of all voxels: are they inside the envelop or not
env_all_vox = full_lattice.flatten()

# all voxels
all_vox_shadow_casting = []

# v_id: voxel id in the list of only interior voxels
v_id = 0

# for all the voxels, place the interiority condition of each voxel in "vox_in"
for vox_in in env_all_vox:
    # if the voxel was outside...
    if vox_in == True:
        # read its value of shadow casting and append it to the list of all voxel shadow casting
        all_vox_shadow_casting.append(vox_shadow_casting[v_id])
        # add one to the voxel id so the next time we read the next voxel
        v_id += 1
    # if the voxel was not inside... 
    else:
        # add 0.0 for its shadow casting
        all_vox_shadow_casting.append(0.0)

# convert to array
shadowcasting_array = np.array(all_vox_shadow_casting)

# reshape to lattice shape
shadowcasting_array = shadowcasting_array.reshape(envelope_lattice.shape)

# convert to lattice
shadowcasting_lattice = tg.to_lattice(shadowcasting_array, envelope_lattice)


print(shadowcasting_lattice.shape)

(15, 9, 4)


### 5.3. Visualize the shadow casting lattice

In [15]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = shadowcasting_lattice.shape
# The bottom left corner of the data set
grid.origin = shadowcasting_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = shadowcasting_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Shadow Casting"] = shadowcasting_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm", clim=[0.5, 1.0],opacity=opacity, shade=True)

# plotting|
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]

## 6. Save Sun Access Lattice into a CSV

In [16]:
# save the shadow casting latice to csv

csv_path = os.path.relpath('../data/shadow_casting_lowres.csv')
shadowcasting_lattice.to_csv(csv_path)

In [17]:
from scipy.interpolate import RegularGridInterpolator

In [18]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/envelope_highres_3.6.csv')
highres_env_lattice = tg.lattice_from_csv(lattice_path)
print(highres_env_lattice.shape)

(43, 24, 11)


In [19]:
def interpolate(low_sunacc_lattice, env_lattice):
    # line spaces
    x_space = np.linspace(low_sunacc_lattice.minbound[0], low_sunacc_lattice.maxbound[0],low_sunacc_lattice.shape[0])
    y_space = np.linspace(low_sunacc_lattice.minbound[1], low_sunacc_lattice.maxbound[1],low_sunacc_lattice.shape[1])
    z_space = np.linspace(low_sunacc_lattice.minbound[2], low_sunacc_lattice.maxbound[2],low_sunacc_lattice.shape[2])

    # interpolation function
    interpolating_function = RegularGridInterpolator((x_space, y_space, z_space), low_sunacc_lattice, bounds_error=False, fill_value=None)

    # high_res lattice
    full_lattice = env_lattice + 1

    # sample points
    sample_points = full_lattice.centroids

    # interpolation
    interpolated_values = interpolating_function(sample_points)

    # lattice construction
    shadowcasting_lattice = tg.to_lattice(interpolated_values.reshape(env_lattice.shape), env_lattice)

    # nulling the unavailable cells
    shadowcasting_lattice *= env_lattice
    
    return shadowcasting_lattice

highres_shadowcasting_lattice = interpolate(shadowcasting_lattice,highres_env_lattice)

In [ ]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

base_lattice = highres_shadowcasting_lattice
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = base_lattice.shape
# The bottom left corner of the data set
grid.origin = base_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Shadow Casting"] = base_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])
p.add_volume(grid, cmap="coolwarm", clim=[0.5, 1.0],opacity=opacity, shade=True)

# plotting
p.show(use_ipyvtk=True)

In [20]:
# save the shadow casting latice to csv
csv_path = os.path.relpath('../data/shadow_casting_highres.csv')
highres_shadowcasting_lattice.to_csv(csv_path)

## 7. Envelope selection

In [21]:
# extra import function
def lattice_from_csv(file_path):
    # read metadata
    meta_df = pd.read_csv(file_path, nrows=3)

    shape = np.array(meta_df['shape'])
    unit = np.array(meta_df['unit'])
    minbound = np.array(meta_df['minbound'])

    # read lattice
    lattice_df = pd.read_csv(file_path, skiprows=5)

    # create the buffer
    buffer = np.array(lattice_df['value']).reshape(shape)

    # create the lattice
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)

    return l

In [22]:
# loading the lattice from csv
shadow_casting_path = os.path.relpath('../data/shadow_casting_highres.csv')
shadow_casting_lattice = lattice_from_csv(shadow_casting_path)

### 7.1. Visualizing the selection

In [23]:
p = pv.Plotter(notebook=True)

base_lattice = shadow_casting_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

def create_mesh(value):

    lattice = np.copy(shadow_casting_lattice)
    lattice[shadow_casting_lattice > value] *= 0.0
    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F")  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([0.001, 1.0])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False, clim=[0.0, 1.0])

    return

p.add_slider_widget(create_mesh, [0, 1], title='', value=1.0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(228.1888554793267, 134.58885319050833, 210.18885392960593),
 (36.00000190734863, -57.60000038146973, 18.00000035762787),
 (0.0, 0.0, 1.0)]

### 7.2. Generating an envelope based on the selection

In [ ]:
#threshed = grid.threshold([0.001, 1.0])
#new_avail_lattice = shadow_casting_lattice < threshold 

lower_bound = 0.01
upper_bound = 0.45
lower_condition = shadow_casting_lattice > lower_bound
upper_condition = shadow_casting_lattice < upper_bound
new_avail_lattice = lower_condition * upper_condition

### 7.3. Visualize the new available lattice

In [ ]:
p = pv.Plotter(notebook=True)

# adding the avilability lattice
new_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

### 7.4. Save new envelope to CSV

In [ ]:
csv_path = os.path.relpath('../data/envelope_highres_3.6_shadow.csv')
new_avail_lattice.to_csv(csv_path)

### 7.5. Low resolution selection

In [ ]:
# loading the lattice from csv
shadow_casting_path = os.path.relpath('../data/shadow_casting_lowres.csv')
shadow_casting_lattice = lattice_from_csv(shadow_casting_path)

In [ ]:
#threshed = grid.threshold([0.001, 1.0])
#new_avail_lattice = shadow_casting_lattice < threshold 

lower_bound = 0.01
upper_bound = 0.45
lower_condition = shadow_casting_lattice > lower_bound
upper_condition = shadow_casting_lattice < upper_bound
new_avail_lattice = lower_condition * upper_condition

### 7.6. Low resolution visualise and save 

In [ ]:
p = pv.Plotter(notebook=True)

# adding the avilability lattice
new_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

In [ ]:
csv_path = os.path.relpath('../data/envelope_lowres_10.8_shadow.csv')
new_avail_lattice.to_csv(csv_path)

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"